# Kinetics Simulator

This notebook allows you to simulate the kinetics of a chemical reaction network.

## Theory

To simulate the time evolution of a system of coupled chemical reactions, we can use mass action kinetics. 

## Setup

Before running any cells, you will need to first create a virtual environment for this notebook. To do this, navigate to the directory containing this notebook and run the following command in your terminal:

<code>conda env create -f environment.yml</code>

Note that you will need to have already installed [anaconda](https://docs.conda.io/projects/conda/en/latest/index.html), a package management system, to sucessfully run this command.

## Running the Notebook

To run a simulation, you will need to define an array of timepoints and three dictionaries. The first dictionary, <code>mass_action_dict</code>, contains all the elementary reactions modeled using mass action kinetics and their corresponding rate constants. The enzymatic reactions modelled by Michaelis-Menten kinetics are contained in the <code>michaelis_menten_dict</code>. The third and final dictionary to define is <code>initial_values</code>, which defines the initial state of the chemical reaction network.

### Constructing the Mass Action Dictionary

A given key-value pair in <code>mass_action_dict</code> represents a chemical reaction and its associated rate constant. A reaction can be either reversible or irreversible, denoted by a <code>'<->'</code>

In [ ]:
%matplotlib osx
import re
import numpy as np
import seaborn as sns
sns.set()
from scipy.integrate import odeint
from collections import OrderedDict
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
from scipy.integrate import odeint

class ChemicalReactionNetwork:
    def __init__(self, mass_action_dict: dict, michaelis_menten_dict: dict, initial_concentrations: dict, time: np.ndarray):
        characters = {'*', '->', '+', '<->'}
        self.species = list(set([specie for specie in sum([i.split(' ') for i in {**mass_action_dict, **michaelis_menten_dict}.keys()], []) if specie not in characters and not specie.isnumeric()]))
        self.mass_action_reactions = self._parse_mass_action(mass_action_dict, self.species) if len(mass_action_dict) > 0 else []
        self.michaelis_menten_reactions = self._parse_michaelis_menten(michaelis_menten_dict, self.species) if len(michaelis_menten_dict) > 0 else []
        self.time = time
        self.initial_concentrations = dict([(specie, 1e-50) if specie not in initial_concentrations.keys() else (specie, initial_concentrations[specie]) for specie in self.species])
        self.concentrations = None

    def _parse_mass_action(self, mass_action_dict: dict, species: list):
        mass_action_reactions = []
        for reaction, rates in mass_action_dict.items():
            rate_name, rate_constant = list(rates.items())[0]
            B, A = np.zeros(len(species)), np.zeros(len(species))
            _substrates, _products = reaction.split('->')
            _substrates, _products = [re.sub(re.compile(r'\s+'), '', sub).split('*') for sub in _substrates.split('+')], [re.sub(re.compile(r'\s+'), '', prod).split('*') for prod in _products.split('+')]
            for _substrate in _substrates:
                substrate = _substrate[1] if len(_substrate) == 2 else _substrate[0]
                stoichiometry_coeff = int(_substrate[0]) if len(_substrate) == 2 else 1
                A[species.index(substrate)] = stoichiometry_coeff
            for _product in _products:
                if _product == ['0']:
                    continue
                product = _product[1] if len(_product) == 2 else _product[0]
                stoichiometry_coeff = int(_product[0]) if len(_product) == 2 else 1
                B[species.index(product)] = stoichiometry_coeff
            mass_action_reactions.append(Reaction(reaction ,B - A, A, rate_name, rate_constant))
        return mass_action_reactions

    def _parse_michaelis_menten(self, michaelis_menten_dict: dict, species: list):
        michaelis_menten_reactions = []
        for reaction, rates in michaelis_menten_dict.items():
            michaelis_constant_keys = list(rates.keys())
            michaelis_constant_keys.sort()
            Km_key, kcat_key = michaelis_constant_keys

            _substrates, _products = reaction.split('->')
            _substrates, _products = [re.sub(re.compile(r'\s+'), '', sub).split('*') for sub in _substrates.split('+')], [re.sub(re.compile(r'\s+'), '', prod).split('*') for prod in _products.split('+')]
            substrates, substrate_stoichiometries, products, product_stoichiometries = [], [], [], []
            for _substrate in _substrates:
                substrate = _substrate[1] if len(_substrate) == 2 else _substrate[0]
                substrates.append(substrate)
                stoichiometry_coeff = int(_substrate[0]) if len(_substrate) == 2 else 1
                substrate_stoichiometries.append(stoichiometry_coeff)
            for _product in _products:
                product = _product[1] if len(_product) == 2 else _product[0]
                products.append(product)
                stoichiometry_coeff = int(_product[0]) if len(_product) == 2 else 1
                product_stoichiometries.append(stoichiometry_coeff)

                    
            enzyme = list(set(substrates).intersection(set(products)))[0]
            substrate = list(set(substrates) - set([enzyme]))[0]
            product = list(set(products) - set([enzyme]))[0]
            substrate_index, enzyme_index, product_index = species.index(substrate), species.index(enzyme), species.index(product)
            substrate_stoichiometry, product_stoichiometry = substrate_stoichiometries[substrates.index(substrate)], product_stoichiometries[products.index(product)]
            michaelis_menten_reactions.append(MichaelisMentenReaction(reaction, substrate_index, substrate_stoichiometry, product_index, product_stoichiometry, enzyme_index, rates[kcat_key], kcat_key, rates[Km_key], Km_key))
        return michaelis_menten_reactions

    def integrate(self):
        N = np.vstack([reaction.N for reaction in self.mass_action_reactions]) if len(self.mass_action_reactions) > 0 else np.array([])
        A = np.vstack([reaction.A for reaction in self.mass_action_reactions]) if len(self.mass_action_reactions) > 0 else np.array([])
        K = np.diag(np.array([reaction.rate for reaction in self.mass_action_reactions])) if len(self.mass_action_reactions) > 0 else np.array([])
        initial_values = np.array([self.initial_concentrations[specie] for specie in self.species])

        def ODEs(concentrations: np.ndarray, time: np.ndarray, N: np.ndarray, A: np.ndarray, K: np.ndarray, michaelis_menten_reactions):
            mass_action_rates = np.dot(N.T, np.dot(K, np.prod(np.power(concentrations, A), axis=1))) if len(N) > 0 else np.zeros(len(concentrations))
            michaelis_menten_rates = np.sum([reaction.compute_velocity(concentrations) for reaction in michaelis_menten_reactions], axis=0) if len(michaelis_menten_reactions) > 0 else np.zeros(len(concentrations))
            return mass_action_rates + michaelis_menten_rates 

        self.concentrations = odeint(ODEs, initial_values, self.time, args=(N, A, K, self.michaelis_menten_reactions))

    def update_parameter(self, param_name: str, new_value: float):
        if param_name in self.initial_concentrations.keys():
            self.initial_concentrations[param_name] = new_value

        elif param_name in set([reaction.rate_name for reaction in self.mass_action_reactions]):
            for reaction in self.mass_action_reactions:
                if param_name == reaction.rate_name:
                    reaction.rate = new_value

        elif param_name in set(sum([[reaction.kcat_name, reaction.Km_name] for reaction in self.michaelis_menten_reactions], [])):
            for reaction in self.michaelis_menten_reactions:
                if param_name == reaction.kcat_name:
                    reaction.kcat = new_value 
                elif param_name == reaction.Km_name:
                    reaction.Km = new_value

class Reaction:
    def __init__(self, scheme: str, N: np.ndarray, A: np.ndarray, rate_name: str, rate: float):
        self.scheme = scheme
        self.N = N
        self.A = A
        self.rate_name = rate_name
        self.rate = rate

class MichaelisMentenReaction:
    def __init__(self, scheme: str, substrate_ind: int, substrate_stoichiometry: int, product_ind: int, product_stoichiometry: int, enzyme_ind: int, kcat: float, kcat_name: str, Km: float, Km_name: str):
        self.scheme = scheme
        self.substrate_ind, self.substrate_stoichiometry = int(substrate_ind), substrate_stoichiometry
        self.product_ind, self.product_stoichiometry = int(product_ind), product_stoichiometry
        self.enzyme_ind = int(enzyme_ind)
        self.kcat, self.kcat_name = kcat, kcat_name
        self.Km, self.Km_name = Km, Km_name

    def compute_velocity(self, concentrations: np.ndarray):
        rates = np.zeros(len(concentrations))
        velocity = (self.kcat * concentrations[self.enzyme_ind] * concentrations[self.substrate_ind]) / (self.Km + concentrations[self.substrate_ind])
        rates[self.substrate_ind] = self.substrate_stoichiometry * -velocity
        rates[self.product_ind] = self.product_stoichiometry * velocity
        return rates

def process_reaction_dict(reaction_dict: dict):
    """
    Splits reversible reactions into elementary reactions.
    """
    updated_reaction_dict = {}
    for reaction in reaction_dict.keys():
        if '<->' in reaction:
            forward_constant, reverse_constant = reaction_dict[reaction].keys()
            split = reaction.split(' ')
            split[split.index('<->')] = '->'
            _forward, _reverse = split, split
            forward, reverse = ' '.join(_forward), ' '.join(_reverse[::-1])
            updated_reaction_dict[forward] = OrderedDict({forward_constant: reaction_dict[reaction][forward_constant]})
            updated_reaction_dict[reverse] = OrderedDict({reverse_constant: reaction_dict[reaction][reverse_constant]})
        else:
            updated_reaction_dict[reaction] = reaction_dict[reaction]
    return updated_reaction_dict
    
def _initialize_plot(plot_kwargs: dict, sliders: set, figsize: tuple):
    fig, ax = plt.subplots(figsize=figsize)
    plt.subplots_adjust(bottom=0.5)
    slider_dict = {}
    for ind, slider in enumerate(sliders):
        _ax = plt.axes([0.25, 0.3 - (0.05 * ind), 0.65, 0.03])
        slider_dict[slider] = Slider(ax=_ax, label=slider, valmin=plot_kwargs[slider]['min'], valmax=plot_kwargs[slider]['max'], valinit=plot_kwargs[slider]['start'], valstep=plot_kwargs[slider]['stepsize'])
    return fig, ax, slider_dict

def plot(reaction_network, plot_kwargs: dict):
    figsize = (8,8) if 'figsize' not in plot_kwargs.keys() else plot_kwargs['figsize']
    title = 'Mass Action Kinetics' if 'title' not in plot_kwargs.keys() else plot_kwargs['title']
    xlabel = 'Time (s)' if 'xlabel' not in plot_kwargs.keys() else plot_kwargs['xlabel']
    ylabel = 'Concentration (nM)' if 'ylabel' not in plot_kwargs.keys() else plot_kwargs['ylabel']
    fontsize = 12 if 'fontsize' not in plot_kwargs.keys() else plot_kwargs['fontsize']
    colormap = None if 'colormap' not in plot_kwargs.keys() else list(sns.color_palette(plot_kwargs['colormap'], n_colors=len(reaction_network.species)).as_hex())

    ma_params = set([x.rate_name for x in reaction_network.mass_action_reactions]) if reaction_network.mass_action_reactions else set()
    mm_params = set(sum([[x.kcat_name, x.Km_name] for x in reaction_network.michaelis_menten_reactions], []))
    sliders = set(plot_kwargs.keys()).intersection(set(reaction_network.species) | ma_params | mm_params)

    fig, ax, slider_dict = _initialize_plot(plot_kwargs, sliders, figsize)
    plot = ax.plot(np.vstack([reaction_network.time] * len(reaction_network.species)).T, reaction_network.concentrations, label=reaction_network.species)
    for i,j in enumerate(ax.lines):
        j.set_color(colormap[i])

    leg = ax.legend(fancybox=True, shadow=True, loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_title(title, fontsize=fontsize)
    ax.set_xlabel(xlabel, fontsize=fontsize)
    ax.set_ylabel(ylabel, fontsize=fontsize)

    lined = {}  # Will map legend lines to original lines.
    for legline, origline in zip(leg.get_lines(), ax.get_lines()):
        legline.set_picker(True)  # Enable picking on the legend line.
        lined[legline] = origline

    def on_pick(event):
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)
        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()
    fig.canvas.mpl_connect('pick_event', on_pick)

    def update(val):
        for slider in sliders:
            reaction_network.update_parameter(slider, slider_dict[slider].val)
        reaction_network.integrate()
        for ind, line in enumerate(plot):
            line.set_ydata(reaction_network.concentrations[:, ind])
        fig.canvas.draw_idle()
    for slider in sliders:
        slider_dict[slider].on_changed(update)
    plt.show()

In [ ]:
time = np.linspace(0,100000,500)
mass_action_dict = {'P -> A': OrderedDict({'k1': 0.0001}),
                    'A -> B': OrderedDict({'k3': 0.0001}),
                    'B -> C': OrderedDict({'k4': 0.0001}),
                    'C -> 2 * D': OrderedDict({'k5': 0.0001})}
michaelis_menten_dict = {}
initial_values = OrderedDict({'A': 1000, 'P': 1000})
plot_kwargs = {'title': 'Mass Action Kinetics of a Model Reaction',
                'xlabel': 'Time (s)',
                'ylabel': 'Concentration (nM)',
                'fontsize': 12,
                'figsize': (12,12),
                'colormap': 'CMRmap',
                'k1': {'min': 1e-50, 'max': 1e-3, 'start': 0.0001, 'stepsize': 1e-6},
                'k3': {'min': 1e-50, 'max': 1e-3, 'start': 0.0001, 'stepsize': 1e-6},
                'k4': {'min': 1e-50, 'max': 1e-3, 'start': 0.0001, 'stepsize': 1e-6},
                'k5': {'min': 1e-50, 'max': 1e-3, 'start': 0.0001, 'stepsize': 1e-6}}

mass_action_dict = process_reaction_dict(mass_action_dict)
reaction_network = ChemicalReactionNetwork(mass_action_dict, michaelis_menten_dict, initial_values, time)
concentrations = reaction_network.integrate()
plot(reaction_network, plot_kwargs)

In [ ]:
time = np.linspace(0,10000,500)
characters = {'*', '->', '+', '<->'}
mass_action_dict = {'GAP + Gsp1:GTP <-> GAP:Gsp1:GTP': OrderedDict({'kon': 2.1e-2, 'koff': 150}),
                'GAP:Gsp1:GTP -> GAP:Gsp1:GDP + Pi': OrderedDict({'kchem_GAP': 9.2}), 
                'Gsp1:GTP -> Gsp1:GDP + Pi': OrderedDict({'kchem': 5.4e-5}),
                'GAP:Gsp1:GDP <-> GAP + Gsp1:GDP': OrderedDict({'koff_prime': 1000000, 'kon_prime': 10})}
michaelis_menten_dict = {'Pi + PNP -> Hypoxanthine + PNP': OrderedDict({'kcat_PNP': 57, 'Km_PNP': 45000}),
                'Hypoxanthine + XO -> H2O2 + XO': OrderedDict({'kcat_XO': 302, 'Km_XO': 151*1000*1000}),
                'H2O2 + HP -> Resorufin + HP': OrderedDict({'kcat_HP': 12.1, 'Km_HP': 2200})}
initial_values = OrderedDict({'GAP': 1,
                'Gsp1:GTP': 5,
                'PNP': 10, 'XO': 10, 'HP':100})
plot_kwargs = {'title': 'GTP Hydrolysis Mass Action Kinetics of Gsp1',
                'xlabel': 'Time (s)',
                'ylabel': 'Concentration (nM)',
                'fontsize': 12,
                'figsize': (12,12),
                'colormap': 'CMRmap',
                'PNP': {'min': 1e-50, 'max': 50000, 'start': 10, 'stepsize': 0.5},
                'XO': {'min': 1e-50, 'max': 50000, 'start': 10, 'stepsize': 0.5},
                'HP': {'min': 1e-50, 'max': 50000, 'start': 100, 'stepsize': 0.5}}

mass_action_dict = process_reaction_dict(mass_action_dict)
reaction_network = ChemicalReactionNetwork(mass_action_dict, michaelis_menten_dict, initial_values, time)
concentrations = reaction_network.integrate()
plot(reaction_network, plot_kwargs)